In [1]:
#!pip install spacy
#!pip install lxml
#!pip install nltk

In [3]:
import bs4 as bs
import urllib.request
import re
import nltk
import numpy as np
import random
import string

import spacy

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /data_lids/home/walter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
print(nltk.__version__)
print(spacy.__version__)

3.7
3.6.0


In [6]:
!python3 -m spacy download es_core_news_sm

2023-07-13 02:42:48.429645: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 20.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


# Load Database

In [7]:
data = urllib.request.urlopen('https://es.wikipedia.org/wiki/Inteligencia_artificial')

In [8]:
data = data.read()
data

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" lang="es" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Inteligencia artificial - Wikipedia, la enciclopedia libre</title>\n<script>document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled";(function(){var cookie=document.c

In [9]:
data_html = bs.BeautifulSoup(data, 'lxml')
data_html

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" dir="ltr" lang="es">
<head>
<meta charset="utf-8"/>
<title>Inteligencia artificial - Wikipedia, la enciclopedia libre</title>
<script>document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled";(function(){var cookie=document.cookie.

In [11]:
parrafos = data_html.find_all('p')
len(parrafos)

68

In [12]:
parrafos[0].text

'La inteligencia artificial  (IA), en el contexto de las ciencias de la computación, es una disciplina y un conjunto de capacidades cognoscitivas e intelectuales expresadas por sistemas informáticos o combinaciones de algoritmos cuyo propósito es la creación de máquinas que imiten la inteligencia humana para realizar tareas, y que pueden mejorar conforme recopilan información.[1]\u200b[2]\u200b A diferencia de la inteligencia sintética, la inteligencia artificial no tiene como finalidad reemplazar a los humanos, sino mejorar significativamente las capacidades y contribuciones de estos. Se hizo presente poco después de la Segunda Guerra Mundial con el desarrollo de la «prueba de Turing», mientras que la locución fue acuñada en 1956 por el informático John McCarthy en la Conferencia de Dartmouth.\n'

In [13]:
contenido = ''

for p in parrafos:
    contenido += p.text

contenido

'La inteligencia artificial  (IA), en el contexto de las ciencias de la computación, es una disciplina y un conjunto de capacidades cognoscitivas e intelectuales expresadas por sistemas informáticos o combinaciones de algoritmos cuyo propósito es la creación de máquinas que imiten la inteligencia humana para realizar tareas, y que pueden mejorar conforme recopilan información.[1]\u200b[2]\u200b A diferencia de la inteligencia sintética, la inteligencia artificial no tiene como finalidad reemplazar a los humanos, sino mejorar significativamente las capacidades y contribuciones de estos. Se hizo presente poco después de la Segunda Guerra Mundial con el desarrollo de la «prueba de Turing», mientras que la locución fue acuñada en 1956 por el informático John McCarthy en la Conferencia de Dartmouth.\nEn la actualidad, la inteligencia artificial abarca una gran variedad de subcampos. Éstos van desde áreas de propósito general, aprendizaje y percepción, a otras más específicas como el reconoc

# Preprocessing

In [14]:
contenido = contenido.lower()
contenido

'la inteligencia artificial  (ia), en el contexto de las ciencias de la computación, es una disciplina y un conjunto de capacidades cognoscitivas e intelectuales expresadas por sistemas informáticos o combinaciones de algoritmos cuyo propósito es la creación de máquinas que imiten la inteligencia humana para realizar tareas, y que pueden mejorar conforme recopilan información.[1]\u200b[2]\u200b a diferencia de la inteligencia sintética, la inteligencia artificial no tiene como finalidad reemplazar a los humanos, sino mejorar significativamente las capacidades y contribuciones de estos. se hizo presente poco después de la segunda guerra mundial con el desarrollo de la «prueba de turing», mientras que la locución fue acuñada en 1956 por el informático john mccarthy en la conferencia de dartmouth.\nen la actualidad, la inteligencia artificial abarca una gran variedad de subcampos. éstos van desde áreas de propósito general, aprendizaje y percepción, a otras más específicas como el reconoc

In [15]:
lista_oraciones = nltk.sent_tokenize(contenido)
len(lista_oraciones)

174

In [16]:
nlp = spacy.load('es_core_news_sm')
nlp

In [17]:
stop_words = spacy.lang.es.stop_words.STOP_WORDS
len(stop_words)

521

In [19]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [20]:
def preprocesamiento(texto):
    # URLs
    texto = re.sub(r'https?://[A-Za-z0-9./]+','', texto)

    # Espacios en blanco
    texto = re.sub(r' +', ' ', texto)

    documento = nlp(texto)
    lista = []
    for token in documento:
        lista.append(token.lemma_)

    lista = [palabra for palabra in lista if palabra not in stop_words and palabra not in string.punctuation]
    lista = ' '.join([str(elemento) for elemento in lista if not elemento.isdigit()])

    return lista

In [21]:
texto_test = 'Hola, Como estas? https://cualquierweb.com'
texto_test = preprocesamiento(texto_test)
texto_test

'Hola Como'

In [23]:
lista_oraciones_preprocesadas = []

for i in range(len(lista_oraciones)):
    lista_oraciones_preprocesadas.append(preprocesamiento(lista_oraciones[i]))

In [24]:
for _ in range(5):
    i = random.randint(0, len(lista_oraciones) - 1)
    print(lista_oraciones[i])
    print(lista_oraciones_preprocesadas[i])
    print('---')

[45]​ de esta manera, se concluyó que los seres humanos rara vez usan la deducción paso a paso que la investigación temprana de la inteligencia artificial seguía; en cambio, resuelven la mayoría de sus problemas utilizando juicios rápidos e intuitivos.
45]​ concluir humano rara deducción paso paso investigación inteligencia artificial seguir cambio resolver mayoría problema utilizar juicio rápido intuitivo
---
en marzo de 2023, cientos de empresarios como elon musk, steve wozniak (cofundador de apple) o los presidentes de numerosas compañías tecnológicas; intelectuales como yuval noah harari y cientos de académicos e investigadores especializados en inteligencia artificial firmaron una carta abierta avisando del peligro de la falta de regulación de la ia, poniendo el foco sobre openai, la empresa que ha desarrollado chatgpt.
marzo ciento empresario elon musk steve wozniak cofundador apple president numeroso compañía tecnológico intelectual yuval noah harari ciento académico investigado

# Chat

In [25]:
textos_bienvenida = ['hey', 'hola']
textos_bienvenida_respuestasa = ['hola', 'hey', ':)', 'que tal?', 'como te ayudo?']

In [27]:
def responder_saludo(texto):
    for palabra in texto.split():
        if palabra.lower() in textos_bienvenida:
            return random.choice(textos_bienvenida_respuestasa)

In [28]:
responder_saludo('hola robot!')

'hey'

# TF-IDF

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
frases_test = lista_oraciones_preprocesadas[:3]
frases_test

['inteligencia artificial ia contexto ciencia computación disciplina conjunto capacidad cognoscitiva intelectual expresado sistema informático combinación algoritmo cuyo propósito creación máquina imitar inteligencia humano tarea mejorar conforme recopilar información',
 '1]\u200b[2]\u200b diferencia inteligencia sintético inteligencia artificial finalidad reemplazar humano mejorar significativamente capacidad contribución',
 'presente guerra mundial desarrollo « prueba turing » locución acuñar informático john mccarthy conferencia dartmouth']

In [31]:
frases_test.append(frases_test[0])
frases_test

['inteligencia artificial ia contexto ciencia computación disciplina conjunto capacidad cognoscitiva intelectual expresado sistema informático combinación algoritmo cuyo propósito creación máquina imitar inteligencia humano tarea mejorar conforme recopilar información',
 '1]\u200b[2]\u200b diferencia inteligencia sintético inteligencia artificial finalidad reemplazar humano mejorar significativamente capacidad contribución',
 'presente guerra mundial desarrollo « prueba turing » locución acuñar informático john mccarthy conferencia dartmouth',
 'inteligencia artificial ia contexto ciencia computación disciplina conjunto capacidad cognoscitiva intelectual expresado sistema informático combinación algoritmo cuyo propósito creación máquina imitar inteligencia humano tarea mejorar conforme recopilar información']

In [32]:
vectores_palabras = TfidfVectorizer()
palabras_vectorizadas = vectores_palabras.fit_transform(frases_test)

In [33]:
type(palabras_vectorizadas)

scipy.sparse._csr.csr_matrix

In [34]:
print(vectores_palabras.get_feature_names_out())

['acuñar' 'algoritmo' 'artificial' 'capacidad' 'ciencia' 'cognoscitiva'
 'combinación' 'computación' 'conferencia' 'conforme' 'conjunto'
 'contexto' 'contribución' 'creación' 'cuyo' 'dartmouth' 'desarrollo'
 'diferencia' 'disciplina' 'expresado' 'finalidad' 'guerra' 'humano' 'ia'
 'imitar' 'información' 'informático' 'intelectual' 'inteligencia' 'john'
 'locución' 'mccarthy' 'mejorar' 'mundial' 'máquina' 'presente'
 'propósito' 'prueba' 'recopilar' 'reemplazar' 'significativamente'
 'sintético' 'sistema' 'tarea' 'turing']


In [35]:
len(vectores_palabras.get_feature_names_out())

45

In [36]:
print(vectores_palabras.vocabulary_)

{'inteligencia': 28, 'artificial': 2, 'ia': 23, 'contexto': 11, 'ciencia': 4, 'computación': 7, 'disciplina': 18, 'conjunto': 10, 'capacidad': 3, 'cognoscitiva': 5, 'intelectual': 27, 'expresado': 19, 'sistema': 42, 'informático': 26, 'combinación': 6, 'algoritmo': 1, 'cuyo': 14, 'propósito': 36, 'creación': 13, 'máquina': 34, 'imitar': 24, 'humano': 22, 'tarea': 43, 'mejorar': 32, 'conforme': 9, 'recopilar': 38, 'información': 25, 'diferencia': 17, 'sintético': 41, 'finalidad': 20, 'reemplazar': 39, 'significativamente': 40, 'contribución': 12, 'presente': 35, 'guerra': 21, 'mundial': 33, 'desarrollo': 16, 'prueba': 37, 'turing': 44, 'locución': 30, 'acuñar': 0, 'john': 29, 'mccarthy': 31, 'conferencia': 8, 'dartmouth': 15}


In [37]:
print(vectores_palabras.idf_)

[1.91629073 1.51082562 1.22314355 1.22314355 1.51082562 1.51082562
 1.51082562 1.51082562 1.91629073 1.51082562 1.51082562 1.51082562
 1.91629073 1.51082562 1.51082562 1.91629073 1.91629073 1.91629073
 1.51082562 1.51082562 1.91629073 1.91629073 1.22314355 1.51082562
 1.51082562 1.51082562 1.22314355 1.51082562 1.22314355 1.91629073
 1.91629073 1.91629073 1.22314355 1.91629073 1.51082562 1.91629073
 1.51082562 1.91629073 1.51082562 1.91629073 1.91629073 1.91629073
 1.51082562 1.51082562 1.91629073]


In [38]:
palabras_vectorizadas.todense()

matrix([[0.        , 0.19281153, 0.15609755, 0.15609755, 0.19281153,
         0.19281153, 0.19281153, 0.19281153, 0.        , 0.19281153,
         0.19281153, 0.19281153, 0.        , 0.19281153, 0.19281153,
         0.        , 0.        , 0.        , 0.19281153, 0.19281153,
         0.        , 0.        , 0.15609755, 0.19281153, 0.19281153,
         0.19281153, 0.15609755, 0.19281153, 0.3121951 , 0.        ,
         0.        , 0.        , 0.15609755, 0.        , 0.19281153,
         0.        , 0.19281153, 0.        , 0.19281153, 0.        ,
         0.        , 0.        , 0.19281153, 0.19281153, 0.        ],
        [0.        , 0.        , 0.20976226, 0.20976226, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.32863312, 0.        , 0.        ,
         0.        , 0.        , 0.32863312, 0.        , 0.        ,
         0.32863312, 0.        , 0.20976226, 0.        , 0.        ,
         0.        , 0.        , 

In [39]:
# Valores del TF-IDF
palabras_vectorizadas.todense().shape

(4, 45)

# Cosine similarity

In [41]:
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
palabras_vectorizadas[0].todense()

matrix([[0.        , 0.19281153, 0.15609755, 0.15609755, 0.19281153,
         0.19281153, 0.19281153, 0.19281153, 0.        , 0.19281153,
         0.19281153, 0.19281153, 0.        , 0.19281153, 0.19281153,
         0.        , 0.        , 0.        , 0.19281153, 0.19281153,
         0.        , 0.        , 0.15609755, 0.19281153, 0.19281153,
         0.19281153, 0.15609755, 0.19281153, 0.3121951 , 0.        ,
         0.        , 0.        , 0.15609755, 0.        , 0.19281153,
         0.        , 0.19281153, 0.        , 0.19281153, 0.        ,
         0.        , 0.        , 0.19281153, 0.19281153, 0.        ]])

In [42]:
cosine_similarity(palabras_vectorizadas[0], palabras_vectorizadas[1])

array([[0.261947]])

In [43]:
cosine_similarity(palabras_vectorizadas[0], palabras_vectorizadas[3])

array([[1.]])

In [44]:
similaridad = cosine_similarity(palabras_vectorizadas[0], palabras_vectorizadas)
similaridad

array([[1.      , 0.261947, 0.028286, 1.      ]])

In [45]:
# Orden de los indices de menor a mayor
similaridad.argsort()

array([[2, 1, 0, 3]])

In [46]:
algo = similaridad.flatten()
algo.sort()
algo[-2]

1.0

In [47]:
# Obtenemos el segundo
i = similaridad.argsort()[0][-2]
i

0

In [46]:
# Pasa a array
i = i.flatten()
i

array([0])

# Responder al usuario

In [48]:
def responder(texto_usuario):
    respuesta_chatbot = ''
    lista_oraciones_preprocesadas.append(texto_usuario)

    tfidf = TfidfVectorizer()
    palabras_vectorizadas = tfidf.fit_transform(lista_oraciones_preprocesadas)

    similaridad = cosine_similarity(palabras_vectorizadas[-1], palabras_vectorizadas)
    indice_oracion = similaridad.argsort()[0][-2]
    vector_similar = similaridad.flatten()
    vector_similar.sort()
    vector_encontrado = vector_similar[-2]

    if (vector_encontrado == 0):
        respuesta_chatbot = respuesta_chatbot + 'Disculpa, no entendi!'
        return respuesta_chatbot
    else:
        respuesta_chatbot = respuesta_chatbot + lista_oraciones[indice_oracion]
        return respuesta_chatbot
    

# Conversar

In [49]:
continuar = True
print('Hola, soy un robot y voy a ayudarte. Digite "salir" para terminar...')
while (continuar == True):
    texto_usuario = input()
    print('---')
    print('Usuario: ' + texto_usuario)
    if (texto_usuario != 'salir'):
        if (responder_saludo(texto_usuario) != None):
            print('Robot: ' + responder_saludo(texto_usuario))
        else:
            print('Robot: ', end='')
            print(responder(preprocesamiento(texto_usuario)))
            lista_oraciones_preprocesadas.remove(preprocesamiento(texto_usuario))
    else:
        continuar = False
        print('Robot: Hasta pronto!')

Hola, soy un robot y voy a ayudarte. Digite "salir" para terminar...
---
Usuario: hola robot
Robot: :)
---
Usuario: 
Robot: Desculpe, não entendi!
---
Usuario: turing
Robot: sin embargo, esta situación no es equivalente a una prueba de turing, que requiere que el participante se encuentre sobre aviso de la posibilidad de hablar con una máquina.
---
Usuario: 
Robot: Desculpe, não entendi!
---
Usuario: salir
Robot: Hasta pronto!
